In [1]:
#Check para ver si lee mi key

import os
from dotenv import load_dotenv
import requests
import pandas as pd
from io import StringIO
import time

# 1) Cargar el archivo .env
load_dotenv()

# 2) Leer la variable del entorno
API_KEY = os.getenv("AEMET_API_KEY")

# 3) Comprobar que se ha cargado
if API_KEY is None:
    print("No se ha encontrado AEMET_API_KEY. Revisa tu archivo .env")
else:
    print("✅ API key cargada correctamente. Longitud:", len(API_KEY))
    # Opcional: mostrar solo los primeros caracteres para comprobar
    print("Inicio de la key:", API_KEY[:10], "... (oculto)")


✅ API key cargada correctamente. Longitud: 281
Inicio de la key: eyJhbGciOi ... (oculto)


In [2]:
# Estación de Malaga (código que ves en la web: 6172X)
STATION_ID_MALAGA = "6172X"

# Endpoint /api/ de climatologías 15/07/25 al 31/07/25:
BASE_URL_MALAGA = "https://opendata.aemet.es/opendata/sh/f1372dc9"


print("✅ Configuración lista para estación", STATION_ID_MALAGA)


✅ Configuración lista para estación 6172X


In [3]:
# evitar errores de tiempos de espera

def get_with_retry(url, *, headers=None, params=None, max_retries=5, base_wait=5, label=""):
    """
    Hace una petición GET con reintentos si recibe 429 Too Many Requests.
    - max_retries: nº máximo de intentos.
    - base_wait: segundos base de espera (se multiplica por el nº de intento).
    """
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, headers=headers, params=params, timeout=30)
        except Exception as e:
            print(f"[{label}] ❌ Error en intento {attempt}: {e}")
            # si es un error de conexión raro, esperamos un poco y reintentamos
            time.sleep(base_wait * attempt)
            continue

        # Si es 429 Too Many Requests → esperamos y reintentamos
        if resp.status_code == 429:
            wait = base_wait * attempt
            print(f"[{label}] ⚠️ 429 Too Many Requests. Esperando {wait} segundos antes de reintentar...")
            time.sleep(wait)
            continue

        # Para cualquier otro código, si es error >400 lanza excepción:
        try:
            resp.raise_for_status()
            return resp  # éxito
        except requests.exceptions.HTTPError as e:
            print(f"[{label}] ❌ HTTPError (código {resp.status_code}): {e}")
            return None

    print(f"[{label}] ❌ Se alcanzó el nº máximo de reintentos ({max_retries})")
    return None

In [4]:
# Función para descargar datos de un año específico del 15 al 31 julio
def get_aemet_july_mid_year(year, station_id=STATION_ID_MALAGA):
    start = f"{year}-07-15T00:00:00UTC"
    end   = f"{year}-07-31T23:59:59UTC"
    url_api = (
        f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/"
        f"fechaini/{start}/fechafin/{end}/estacion/{station_id}?api_key={API_KEY}"
    )

    # 1️⃣ Obtener URL de datos
    resp = get_with_retry(url_api, label=f"{station_id} {year}")
    if resp is None:
        return pd.DataFrame()
    
    data_url = resp.json().get("datos")
    if not data_url:
        print(f"[{year}] ❌ No se encontró URL de datos")
        return pd.DataFrame()

    # 2️⃣ Descargar datos finales
    r_data = get_with_retry(data_url, label=f"{station_id} DATOS {year}")
    if r_data is None:
        return pd.DataFrame()
    
    # 3️⃣ Cargar JSON a DataFrame
    try:
        data_json = r_data.json()
        df = pd.DataFrame(data_json)
        df["year"] = year
        # Convertir fecha a datetime
        if "fecha" in df.columns:
            df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
        print(f"[{year}] ✅ Datos descargados: {len(df)} filas")
        return df
    except Exception as e:
        print(f"[{year}] ❌ Error procesando JSON: {e}")
        return pd.DataFrame()

# 5️⃣ Descargar y unir los tres años
df_july_malaga = pd.concat(
    [get_aemet_july_mid_year(y) for y in [2023, 2024, 2025]],
    ignore_index=True
)

# 6️⃣ Ordenar por fecha
df_july_malaga = df_july_malaga.sort_values("fecha").reset_index(drop=True)

# 7️⃣ Mostrar resultados
print(df_july_malaga.head())
print(f"Total de registros: {len(df_july_malaga)}")

[2023] ✅ Datos descargados: 17 filas
[2024] ✅ Datos descargados: 17 filas
[2025] ✅ Datos descargados: 17 filas
       fecha indicativo  nombre provincia altitud  tmed prec  tmin horatmin  \
0 2023-07-15      6172X  MÁLAGA    MALAGA      25  28,5  0,0  24,4    04:10   
1 2023-07-16      6172X  MÁLAGA    MALAGA      25  29,8  0,0  26,2    23:28   
2 2023-07-17      6172X  MÁLAGA    MALAGA      25  27,4  0,0  24,9    04:29   
3 2023-07-18      6172X  MÁLAGA    MALAGA      25  28,5  0,0  25,9    23:54   
4 2023-07-19      6172X  MÁLAGA    MALAGA      25  30,9  0,0  25,4    04:16   

   tmax  ... dir velmedia racha horaracha hrMedia hrMax horaHrMax hrMin  \
0  32,6  ...  15      2,5   7,2     18:40      66    87     03:10    37   
1  33,5  ...  33      2,2   7,5     07:20      49    83     23:40    32   
2  30,0  ...  24      1,9   5,3     12:50      77    88    Varias    64   
3  31,1  ...  04      2,5   7,8     02:20      69    87     23:50    63   
4  36,4  ...  16      2,5   8,3     15:

In [5]:
df_july_malaga.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,dir,velmedia,racha,horaracha,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,year
0,2023-07-15,6172X,MÁLAGA,MALAGA,25,"28,5","0,0","24,4",04:10,"32,6",...,15,"2,5","7,2",18:40,66,87,03:10,37,21:40,2023
1,2023-07-16,6172X,MÁLAGA,MALAGA,25,"29,8","0,0","26,2",23:28,"33,5",...,33,"2,2","7,5",07:20,49,83,23:40,32,09:40,2023
2,2023-07-17,6172X,MÁLAGA,MALAGA,25,"27,4","0,0","24,9",04:29,"30,0",...,24,"1,9","5,3",12:50,77,88,Varias,64,01:40,2023
3,2023-07-18,6172X,MÁLAGA,MALAGA,25,"28,5","0,0","25,9",23:54,"31,1",...,04,"2,5","7,8",02:20,69,87,23:50,63,04:00,2023
4,2023-07-19,6172X,MÁLAGA,MALAGA,25,"30,9","0,0","25,4",04:16,"36,4",...,16,"2,5","8,3",15:00,52,88,00:30,35,Varias,2023


In [6]:
# Estación de Ibiza (código que ves en la web: B957)
STATION_ID_IBIZA = "B957"

# Endpoint /api/ de climatologías 15/07/25 al 31/07/25:
BASE_URL_IBIZA = "https://opendata.aemet.es/opendata/sh/5c00848c"


print("✅ Configuración lista para estación", STATION_ID_IBIZA)

✅ Configuración lista para estación B957


In [7]:
# Función para descargar datos de un año específico del 15 al 31 julio
def get_aemet_july_mid_year(year, station_id=STATION_ID_IBIZA):
    start = f"{year}-07-15T00:00:00UTC"
    end   = f"{year}-07-31T23:59:59UTC"
    url_api = (
        f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/"
        f"fechaini/{start}/fechafin/{end}/estacion/{station_id}?api_key={API_KEY}"
    )

    # 1️⃣ Obtener URL de datos
    resp = get_with_retry(url_api, label=f"{station_id} {year}")
    if resp is None:
        return pd.DataFrame()
    
    data_url = resp.json().get("datos")
    if not data_url:
        print(f"[{year}] ❌ No se encontró URL de datos")
        return pd.DataFrame()

    # 2️⃣ Descargar datos finales
    r_data = get_with_retry(data_url, label=f"{station_id} DATOS {year}")
    if r_data is None:
        return pd.DataFrame()
    
    # 3️⃣ Cargar JSON a DataFrame
    try:
        data_json = r_data.json()
        df = pd.DataFrame(data_json)
        df["year"] = year
        # Convertir fecha a datetime
        if "fecha" in df.columns:
            df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
        print(f"[{year}] ✅ Datos descargados: {len(df)} filas")
        return df
    except Exception as e:
        print(f"[{year}] ❌ Error procesando JSON: {e}")
        return pd.DataFrame()

# 5️⃣ Descargar y unir los tres años
df_july_ibiza = pd.concat(
    [get_aemet_july_mid_year(y) for y in [2023, 2024, 2025]],
    ignore_index=True
)

# 6️⃣ Ordenar por fecha
df_july_ibiza = df_july_ibiza.sort_values("fecha").reset_index(drop=True)

# 7️⃣ Mostrar resultados
print(df_july_ibiza.head())
print(f"Total de registros: {len(df_july_ibiza)}")

[2023] ✅ Datos descargados: 17 filas
[2024] ✅ Datos descargados: 17 filas
[B957 2025] ⚠️ 429 Too Many Requests. Esperando 5 segundos antes de reintentar...
[B957 2025] ⚠️ 429 Too Many Requests. Esperando 10 segundos antes de reintentar...
[B957 2025] ⚠️ 429 Too Many Requests. Esperando 15 segundos antes de reintentar...
[B957 2025] ⚠️ 429 Too Many Requests. Esperando 20 segundos antes de reintentar...
[2025] ✅ Datos descargados: 17 filas
       fecha indicativo   nombre      provincia altitud  tmed prec  tmin  \
0 2023-07-15       B957  EIVISSA  ILLES BALEARS       3  28,6  0,0  24,5   
1 2023-07-16       B957  EIVISSA  ILLES BALEARS       3  28,4  0,0  23,6   
2 2023-07-17       B957  EIVISSA  ILLES BALEARS       3  29,0  0,0  23,8   
3 2023-07-18       B957  EIVISSA  ILLES BALEARS       3  30,1  0,0  24,3   
4 2023-07-19       B957  EIVISSA  ILLES BALEARS       3  29,7  0,0  23,4   

  horatmin  tmax  ... presMax horaPresMax presMin horaPresMin hrMedia hrMax  \
0    05:20  32,8  ... 

In [8]:
df_july_ibiza.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,year
0,2023-07-15,B957,EIVISSA,ILLES BALEARS,3,"28,6","0,0","24,5",05:20,"32,8",...,"1013,1",00,"1011,2",Varias,67,84,05:20,55,10:50,2023
1,2023-07-16,B957,EIVISSA,ILLES BALEARS,3,"28,4","0,0","23,6",05:00,"33,2",...,"1016,0",23,"1011,0",02,71,89,Varias,61,12:30,2023
2,2023-07-17,B957,EIVISSA,ILLES BALEARS,3,"29,0","0,0","23,8",04:30,"34,1",...,"1017,9",11,"1014,5",03,73,90,04:30,59,13:30,2023
3,2023-07-18,B957,EIVISSA,ILLES BALEARS,3,"30,1","0,0","24,3",04:30,"35,9",...,"1017,6",00,"1014,3",20,72,91,Varias,42,10:30,2023
4,2023-07-19,B957,EIVISSA,ILLES BALEARS,3,"29,7","0,0","23,4",05:30,"36,0",...,"1014,9",00,"1012,2",19,62,83,23:50,49,14:20,2023


In [9]:
# Estación de Coruña (código que ves en la web: 1387)
STATION_ID_CORUÑA = "1387"

# Endpoint /api/ de climatologías 15/07/25 al 31/07/25:
BASE_URL_CORUÑA = "https://opendata.aemet.es/opendata/sh/575fd86a"


print("✅ Configuración lista para estación", STATION_ID_CORUÑA)

✅ Configuración lista para estación 1387


In [10]:
# Función para descargar datos de un año específico del 15 al 31 julio
def get_aemet_july_mid_year(year, station_id=STATION_ID_CORUÑA):
    start = f"{year}-07-15T00:00:00UTC"
    end   = f"{year}-07-31T23:59:59UTC"
    url_api = (
        f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/"
        f"fechaini/{start}/fechafin/{end}/estacion/{station_id}?api_key={API_KEY}"
    )

    # 1️⃣ Obtener URL de datos
    resp = get_with_retry(url_api, label=f"{station_id} {year}")
    if resp is None:
        return pd.DataFrame()
    
    data_url = resp.json().get("datos")
    if not data_url:
        print(f"[{year}] ❌ No se encontró URL de datos")
        return pd.DataFrame()

    # 2️⃣ Descargar datos finales
    r_data = get_with_retry(data_url, label=f"{station_id} DATOS {year}")
    if r_data is None:
        return pd.DataFrame()
    
    # 3️⃣ Cargar JSON a DataFrame
    try:
        data_json = r_data.json()
        df = pd.DataFrame(data_json)
        df["year"] = year
        # Convertir fecha a datetime
        if "fecha" in df.columns:
            df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
        print(f"[{year}] ✅ Datos descargados: {len(df)} filas")
        return df
    except Exception as e:
        print(f"[{year}] ❌ Error procesando JSON: {e}")
        return pd.DataFrame()

# 5️⃣ Descargar y unir los tres años
df_july_coruña = pd.concat(
    [get_aemet_july_mid_year(y) for y in [2023, 2024, 2025]],
    ignore_index=True
)

# 6️⃣ Ordenar por fecha
df_july_coruña = df_july_coruña.sort_values("fecha").reset_index(drop=True)

# 7️⃣ Mostrar resultados
print(df_july_coruña.head())
print(f"Total de registros: {len(df_july_coruña)}")

[1387 2023] ⚠️ 429 Too Many Requests. Esperando 5 segundos antes de reintentar...
[1387 2023] ⚠️ 429 Too Many Requests. Esperando 10 segundos antes de reintentar...
[1387 2023] ⚠️ 429 Too Many Requests. Esperando 15 segundos antes de reintentar...
[2023] ✅ Datos descargados: 17 filas
[2024] ✅ Datos descargados: 17 filas
[2025] ✅ Datos descargados: 17 filas
       fecha indicativo    nombre provincia altitud  tmed prec  tmin horatmin  \
0 2023-07-15       1387  A CORUÑA  A CORUÑA      57  19,8  2,2  16,6    04:50   
1 2023-07-16       1387  A CORUÑA  A CORUÑA      57  19,3  0,0  15,7    05:40   
2 2023-07-17       1387  A CORUÑA  A CORUÑA      57  19,0  0,0  14,4    04:50   
3 2023-07-18       1387  A CORUÑA  A CORUÑA      57  20,7  0,0  17,2    05:20   
4 2023-07-19       1387  A CORUÑA  A CORUÑA      57  20,2  0,0  17,8    05:30   

   tmax  ... presMax horaPresMax presMin horaPresMin hrMedia hrMax horaHrMax  \
0  23,0  ...  1013,2          21  1005,7          04      65    89     08:

In [11]:
df_july_coruña.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,year
0,2023-07-15,1387,A CORUÑA,A CORUÑA,57,"19,8","2,2","16,6",04:50,"23,0",...,"1013,2",21,"1005,7",04,65,89,08:00,52,13:10,2023
1,2023-07-16,1387,A CORUÑA,A CORUÑA,57,"19,3","0,0","15,7",05:40,"22,9",...,"1014,0",22,"1011,4",04,66,79,23:40,52,13:10,2023
2,2023-07-17,1387,A CORUÑA,A CORUÑA,57,"19,0","0,0","14,4",04:50,"23,6",...,"1013,8",00,"1008,7",Varias,72,86,04:00,59,10:20,2023
3,2023-07-18,1387,A CORUÑA,A CORUÑA,57,"20,7","0,0","17,2",05:20,"24,2",...,"1015,1",21,"1009,3",02,82,92,04:30,68,10:30,2023
4,2023-07-19,1387,A CORUÑA,A CORUÑA,57,"20,2","0,0","17,8",05:30,"22,6",...,"1014,9",08,"1012,2",24,78,92,Varias,69,12:10,2023


In [12]:
# Unimos los tres Dataframe en uno
df_climatologia = pd.concat([df_july_malaga, df_july_ibiza, df_july_coruña], ignore_index=True)

In [ ]:
# Exportar a CSV
df_climatologia.to_csv("aemet_abordo.csv", index=False)

In [ ]:
df_climatologia.to_excel("aemet_abordo.xlsx", index=False)

In [25]:
# Creamos copia del dataframe
df_clima = df_climatologia.copy()

In [26]:
df_clima.dtypes

fecha          datetime64[ns]
indicativo             object
nombre                 object
provincia              object
altitud                object
tmed                  float64
prec                   object
tmin                  float64
horatmin               object
tmax                  float64
horatmax               object
dir                    object
velmedia              float64
racha                 float64
horaracha              object
hrMedia                object
hrMax                  object
horaHrMax              object
hrMin                  object
horaHrMin              object
year                    int64
presMax                object
horaPresMax            object
presMin                object
horaPresMin            object
sol                    object
dtype: object

In [27]:
# Columnas numéricas con comas
cols_numericas = ["tmed", "tmin", "tmax", "velmedia", "racha"]

# Reemplazar comas por puntos y convertir a float
df_climatologia[cols_numericas] = df_climatologia[cols_numericas].replace(",", ".", regex=True).astype(float)

# Renombrar columnas a nombres definitivos
df_clima = df_climatologia.rename(columns={
    "tmed": "temperatura_med",
    "tmin": "temperatura_min",
    "tmax": "temperatura_max",
    "dir": "direccion_viento",
    "velmedia": "velocidad_med_viento",
    "racha": "racha_viento",
    "year": "año"
})

In [28]:
# Verificacion
df_clima[["temperatura_med", "temperatura_min", "temperatura_max",
          "velocidad_med_viento", "racha_viento"]].head()

,temperatura_med,temperatura_min,temperatura_max,velocidad_med_viento,racha_viento
0,28.5,24.4,32.6,2.5,7.2
1,29.8,26.2,33.5,2.2,7.5
2,27.4,24.9,30.0,1.9,5.3
3,28.5,25.9,31.1,2.5,7.8
4,30.9,25.4,36.4,2.5,8.3


In [29]:
# Columnas a borrar
cols_a_borrar = [
    "indicativo", "provincia", "altitud", "prec",
    "temperatura_min", "horatmin", "horatmax", "horaracha",
    "hrMedia", "hrMax", "horaHrMax", "hrMin",
    "horaHrMin", "presMax", "horaPresMax",
    "presMin", "horaPresMin", "sol"
]

# Borrar columnas
df_clima = df_clima.drop(columns=cols_a_borrar)


In [31]:
df_clima = df_clima.rename(columns={"nombre": "provincia"})

In [35]:
df_clima["direccion_viento"] = df_clima["direccion_viento"].astype(int)

In [36]:
df_clima.dtypes

fecha                   datetime64[ns]
provincia                       object
temperatura_med                float64
temperatura_max                float64
direccion_viento                 int64
velocidad_med_viento           float64
racha_viento                   float64
año                              int64
dtype: object

In [37]:
df_clima.head()

,fecha,provincia,temperatura_med,temperatura_max,direccion_viento,velocidad_med_viento,racha_viento,año
0,2023-07-15,MÁLAGA,28.5,32.6,15,2.5,7.2,2023
1,2023-07-16,MÁLAGA,29.8,33.5,33,2.2,7.5,2023
2,2023-07-17,MÁLAGA,27.4,30.0,24,1.9,5.3,2023
3,2023-07-18,MÁLAGA,28.5,31.1,4,2.5,7.8,2023
4,2023-07-19,MÁLAGA,30.9,36.4,16,2.5,8.3,2023


In [39]:
# Exportar a CSV
df_clima.to_csv("aemet_clima_todos-a-abordo.csv", index=False)